In [1]:
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon
import seaborn as sns
from h3 import h3

# Import data and convert to geopandas df

In [2]:
trips = pd.read_csv('../data/processed/trips.csv')

## Create geopandas df for start locations

In [3]:
geo_trips_start = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.start_lng, trips.start_lat)).copy()

geo_trips_start.head(2)

,bike,bike_type,identification,start_time,end_time,weekend,duration_sec,start_lng,start_lat,end_lng,end_lat,start_place,end_place,start_plz,end_plz,geometry
0,20507,29,14776184,2019-04-21 17:34:00,2019-04-21 17:47:00,1,780.0,8.884911,53.078923,8.899906,53.078711,2985.0,0.0,28329,28327,POINT (8.88491 53.07892)
1,20507,29,10278649,2019-04-23 17:55:00,2019-04-23 20:13:00,0,8280.0,8.884911,53.078923,8.884911,53.078923,2985.0,2985.0,28329,28329,POINT (8.88491 53.07892)


## Import geodata of Bremen (PLZ based)
https://public.opendatasoft.com/explore/dataset/postleitzahlen-deutschland/table/?refine.note=Bremen

In [4]:
plz_df = gpd.read_file('../data/external/plz_bremen.geojson')

plz_df.head(2)

,note,plz,geometry
0,Bremen,28199,"POLYGON ((8.76410 53.06703, 8.77230 53.06924, ..."
1,Bremen,28309,"POLYGON ((8.86527 53.03539, 8.86535 53.03602, ..."


## Visualization of plz areas of Bremen

Note: 28329 comprises two disjunct areas - anyway the task is to count starts and end per PLZ

In [5]:
plz_bremen = folium.Map(location=[53.113780905428636,8.72900247893676], zoom_start=11)

folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(plz_bremen)

folium.Choropleth(geo_data=plz_df).add_to(plz_bremen)

folium.LayerControl().add_to(plz_bremen)

plz_bremen

# Create new data frame with plz, the corresponding geometry and number of trips started and ended in this area

## Starts

In [6]:
def get_trips_per_plz_df(trips_df, plz_df):

    trips_df['count'] = 1

    dfsjoin = gpd.sjoin(plz_df,trips_df[['geometry','count']])

    dfpivot = pd.pivot_table(dfsjoin,index='plz',columns='count',aggfunc={'count':len})

    dfpivot.columns = dfpivot.rename(columns={1:'trips_count'}).columns.droplevel()

    trips_per_plz_df = plz_df.merge(dfpivot, how='left',on='plz')
    
    return trips_per_plz_df

In [ ]:
plz_starts = get_trips_per_plz_df(geo_trips_start, plz_df)

/Users/timockenga/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [ ]:
def visualize_trips_per_plz_1(plz_starts, aliases, legend_name):

    # Can be used for checking individual quantiles.
    myscale = (plz_starts['trips_count'].quantile((0,0.6,0.75,0.9,0.98,1))).tolist()

    m = folium.Map(location=[53.113780905428636,8.72900247893676], zoom_start=11)
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

    folium.Choropleth(
        geo_data=plz_starts,
        name='choropleth',
        data=plz_starts,
        columns=['plz','trips_count'],
        key_on="feature.properties.plz",
        fill_color='YlGnBu',
        #threshold_scale=myscale,
        fill_opacity=1,
        line_opacity=0.5,
        legend_name=legend_name
    ).add_to(m)

    folium.LayerControl().add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff',
                                'color':'#ffffff', 
                                'fillOpacity': 0, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.5, 
                                    'weight': 0.1}
    popup = folium.features.GeoJson(
        plz_starts,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['plz','trips_count'],
            aliases=aliases,
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )

    m.add_child(popup)
    m.keep_in_front(popup)

    return m

In [ ]:
def visualize_trips_per_plz_2(plz_starts, aliases):

    colormap = cm.linear.YlGnBu_09.scale(0, plz_starts.trips_count.max())

    mymap = folium.Map(location=[53.113780905428636,8.72900247893676], zoom_start=11)
    
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
    
    colormap.caption = aliases[1]
    
    style_function = lambda x: {"weight":0.5, 
                                'color':'black',
                                'fillColor':colormap(x['properties']['trips_count']), 
                                'fillOpacity':1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    
    trips_count=folium.features.GeoJson(
            plz_starts,
            style_function=style_function,
            control=False,
            highlight_function=highlight_function,
            tooltip=folium.features.GeoJsonTooltip(fields=['plz','trips_count'],
                aliases=aliases,
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
                sticky=True
            )
        )
    colormap.add_to(mymap)
    mymap.add_child(trips_count)
    
    return mymap

## Visualizations of starts per plz of all available data of the year 2019

In [ ]:
visualize_trips_per_plz_2(plz_starts, ['Postleitzahl: ', 'Number of starts in 2019: '])

### Save to reports

In [ ]:
visualize_trips_per_plz_2(plz_starts, ['Postleitzahl: ', 'Number of starts in 2019: ']).save('../reports/figures/starts_per_plz_2019.html')

- most trips in august start around the center of Bremen close to the main station

### Check the number of trips per month

In [ ]:
trips['start_time'] = pd.to_datetime(trips['start_time'])
num_trips_month = []
for i in range(1,13):
    num_trips_month.append(len(trips[trips.set_index('start_time').index.month == i]))
ax = sns.barplot(x=np.arange(1,13), y=num_trips_month, palette="Blues_d")
ax.set(ylabel='Number of bookings', xlabel='month of the year 2019') 
ax.set_title('Number of bookings per month (2019)')

## Visualizations of trips per plz of all available data of august 2019

In [ ]:
geo_trips_start['start_time'] = pd.to_datetime(geo_trips_start['start_time'])

In [ ]:
geo_trips_start_august = geo_trips_start[geo_trips_start.set_index('start_time').index.month == 8]

In [ ]:
plz_starts_august = get_trips_per_plz_df(geo_trips_start_august, plz_df)

In [ ]:
# Replace null values with 0
plz_starts_august['trips_count'] = pd.DataFrame(plz_starts_august['trips_count']).fillna(0)

In [ ]:
visualize_trips_per_plz_2(plz_starts_august,['Postleitzahl: ', 'Number of starts in august 2019: '])

- like for the whole year 2019 most trips in august start around the center of Bremen close to the main station

### Save to reports

In [ ]:
visualize_trips_per_plz_2(plz_starts_august,['Postleitzahl: ', 'Number of starts in august 2019: ']).save('../reports/figures/starts_per_plz_august_2019.html')

# Visualization of starts and ends per hexagon resolution 9

### Add h3 hexagon (resolution 9) addresses to trips data frame

In [ ]:
resolution = 9

sHex = []
eHex = []

for index, row in trips.iterrows():
    sHex.append(h3.geo_to_h3(row['start_lat'], row['start_lng'], resolution))
    eHex.append(h3.geo_to_h3(row['end_lat'], row['end_lng'], resolution))

In [ ]:
trips['sHex'] = sHex
trips['eHex'] = eHex

In [ ]:
trips.head(2)

## Visualize start locations

In [ ]:
trips['count'] = 1 # Add count column to count number of trips per hexagon

starts = trips.groupby(['sHex']).agg({'count':'sum'}) # Aggregate all starts related to the corresponding hexagon

starts.reset_index(inplace=True)

There are many hexagons with only one trip in 2019, so we visualize only hexagons with at least 50 trips in 2019.

In [ ]:
starts.sort_values(by='count')

## Functions for visualization

In [ ]:
# Function that takes a list of and latitude and longitude coordinates reverses them and returns a list.
def reverse_lat_lon(hex_coords):
    geom_hex = []

    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1], lat_lon[0]])

    return geom_hex

# Function that draws a polygon onto a folium map.
def add_poly_to_map(polygon, m, color, fillColor, weight, colormap):
    
    style_function = lambda x: {"weight":0, 
                                'color':'black',
                                'fillColor':colormap(weight), 
                                'fillOpacity':0.4}
    
    popup = folium.Popup('Number of trips in 2019: ' + str(weight))
    
    gj = folium.GeoJson(polygon, style_function=style_function)
    
    gj.add_child(popup)
    
    gj.add_to(m)

# Function that draws one or multiple h3 hexagon onto a folium map.
def add_hex_to_map(hex_address, m, color, fillColor, weight, colormap):
    if isinstance(hex_address, str):
        hex_poly = Polygon(reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=hex_address)))

        add_poly_to_map(hex_poly, m, color, fillColor, weight, colormap)

    if isinstance(hex_address, list):

        for address in hex_address:
            hex_poly = Polygon(reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=address)))

            add_poly_to_map(hex_poly, m, color, fillColor, weight, colormap)

## Visualize starts per hexagon in 2019 (at least 50)

In [ ]:
# Create map
m = folium.Map(location=[53.0792962,8.8016937], zoom_start=12)

# Add style elements to map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
colormap = cm.LinearColormap(colors=['blue','red'], index=[50,3320],vmin=0,vmax=3320) # Create colormap
colormap.caption = "number of starts in 2019"
colormap.add_to(m) # Add colormap to map

# Add hexagons with more than 49 to map
for index, row in starts.sort_values(by='count')[:len(starts)-1][starts['count'] > 49].iterrows():
    weight = row['count']
    add_hex_to_map(row['sHex'], m, 'royalblue', 'royalblue', weight, colormap)

# Add hexagon with most trips to the map - seperately for nicer visualization
add_hex_to_map('891f1559917ffff', m, 'royalblue', 'royalblue', 8839, colormap)

# Save map to reports
m.save('../reports/figures/start_locations_hexmap_2019.html')

# Show map
m

- one can see that most trips starts around the city center as the plot shows only hexagons with at least 50 starts in 2019
- we can clearly identify the main station as the area with the most starts

## Visualize end locations

In [ ]:
ends = trips.groupby(['eHex']).agg({'count':'sum'}) # Aggregate all ends related to the corresponding hexagon

In [ ]:
ends.reset_index(inplace=True)

There are many hexagons with only one trip in 2019, so we visualize only hexagons with at least 50 trips in 2019.

In [ ]:
ends.sort_values(by='count')

# Visualize ends per hexagon in 2019 (at least 50)

In [ ]:
# Create map
m = folium.Map(location=[53.0792962,8.8016937], zoom_start=12) # Create map

# Add style elements to map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
colormap = cm.LinearColormap(colors=['blue','red'], index=[50,3260],vmin=0,vmax=3260) # Create colormap
colormap.caption = "number of ends in 2019"
colormap.add_to(m) # Add colormap to map

# Add hexagons with more than 49 to map
for index, row in ends.sort_values(by='count')[:len(ends)-1][ends['count'] >= 50].iterrows():
    weight = row['count']
    add_hex_to_map(row['eHex'], m, 'royalblue', 'royalblue', weight, colormap)

# Add hexagon with most trips to the map - seperately for nicer visualization
add_hex_to_map('891f1559917ffff', m, 'royalblue', 'royalblue', 8315, colormap)

# Save map to reports
m.save('../reports/figures/end_locations_hexmap_2019.html')

# Show map
m

- start and end locations are almost the same regarding the number of trips related to them.
- Genereally, there are some more end locations (1110 - 1088 = 22) than start locations (for hexagons with resolution of 9) that have at least 50 related trips in 2019.